<img src="https://upload.wikimedia.org/wikipedia/commons/f/ff/Uexternado.jpg" width="240" height="240" align="right"/>

# Modelos Supervisados

## Random Forest y Voting

Andrés Martínez<br>
Julio 2024

## Métodos de Ensamble

El objetivo de los métodos de ensamble es combinar diferentes clasificadores en un metaclasificador que tenga un mejor rendimiento de generalización que cada clasificador individual solo. Por ejemplo, suponiendo que recopilamos predicciones de 10 expertos, los métodos de ensamble nos permitirían combinar estratégicamente esas predicciones de los 10 expertos para llegar a una predicción que fuera más precisa y sólida que las predicciones de cada experto individual. Como verá más adelante en este capítulo, existen varios enfoques diferentes para crear un ensamble de clasificadores. Esta sección presentará una explicación básica de cómo funcionan los ensambles y por qué se les suele reconocer por producir un buen desempeño de generalización.

Los métodos de ensamble más populares que utilizan el principio de votación por mayoría. La votación por mayoría simplemente significa que seleccionamos la etiqueta de clase que ha sido predicha por la mayoría de los clasificadores, es decir, recibió más del 50 por ciento de los votos. En sentido estricto, el término
"voto mayoritario" se refiere únicamente a la configuración de clases binarias. Sin embargo, es fácil generalizar el principio de votación por mayoría a entornos multiclase, lo que se conoce como votación por pluralidad.

![Picture title](image-20220726-143842.png)

En la figura se observa como funcionan los métodos de ensamble, primero se generan lo métodos de clasificación, con nuevos datos, luego se obtienen las predicciones y finalmente se generan los votos para obtener el mejor clasificador.


La siguiente función usa un modelo binomial para determinar la probabilidad del error en los métodos de ensamble.


$$ P(y\geq k)= \sum_{k}^{n} \binom{n}{k} \epsilon^{k} (1-\epsilon)^{n-k}=\epsilon_{ensamble}$$

¿Cómo es posible que esta técnica funcione? La siguiente analogía puede ayudar a aclarar este misterio. Supongamos que tenemos una moneda ligeramente sesgada que tiene un 51% de posibilidades de salir cara y 49% de salir cruz. Si la lanza 1.000 veces, generalmente obtendrá más o menos 510 caras y 490 colas, y por tanto una mayoría de caras. Si hace los cálculos, descubrirá que la probabilidad de obtener una mayoría de caras después de 1.000 lanzamientos se acerca al 75%. Cuanto más lances la moneda, mayor será la probabilidad (por ejemplo, con 10.000 lanzamientos, la probabilidad supera el 97%). Esto se debe a la ley de los grandes números: a medida que se sigue lanzando la moneda, la proporción de caras se acerca cada vez más a la probabilidad de cara (51%). La figura siguiente muestra 10 series de lanzamientos de moneda sesgada. Puede ver que, a medida que aumenta el número de lanzamientos, la proporción de caras se acerca al 51%. Finalmente, las 10 series terminan tan cerca del 51% que se sitúan sistemáticamente por encima del 50%.

Del mismo modo, suponga que construye un conjunto que contiene 1.000 clasificadores que, individualmente, sólo aciertan el 51% de las veces (apenas mejor que la adivinación aleatoria). Si predice la clase mayoritariamente votada, puede esperar una precisión de hasta el 75%. Sin embargo, esto sólo es cierto si todos los clasificadores son perfectamente independientes, cometiendo errores no correlacionados, lo cual no es que, evidentemente, no es el caso, ya que se han entrenado con los mismos datos. Es probable que cometan los mismos mismos tipos de errores, por lo que habrá muchos votos mayoritarios para la clase equivocada, reduciendo la precisión del conjunto.

In [ ]:
import numpy as np
import matplotlib as plt
heads_proba = 0.51
coin_tosses = (np.random.rand(10000, 10) < heads_proba).astype(np.int32)
cumulative_heads_ratio = np.cumsum(coin_tosses, axis=0) / np.arange(1, 10001).reshape(-1, 1)

La función nos dice que si tomamos 11 clasificadores con una tasa de error promedio de 0.25, entonces la tasa de error del ensamble es 0.034, mucho menor que escoger un solo clasificador.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
error_range = np.arange(0.0, 1.01, 0.01)
ens_errors = [ensemble_error(n_classifier=11, error=error)
    for error in error_range]
plt.plot(error_range,ens_errors,label='Ensemble error',linewidth=2)
plt.plot(error_range, error_range,linestyle='--', label='Base error',linewidth=2)
plt.xlabel('Base error')
plt.ylabel('Base/Ensemble error')
plt.legend(loc='upper left')
plt.grid(alpha=0.5)
plt.show()

In [ ]:
from scipy.special import comb
import math
def ensemble_error(n_classifier, error):
    k_start = int(math.ceil(n_classifier / 2.))
    probs = [comb(n_classifier, k) * error**k * (1-error)**(n_classifier - k)
        for k in range(k_start, n_classifier + 1)]
    return sum(probs)

ensemble_error(n_classifier=5, error=0.25)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.figure(figsize=(8,3.5))
plt.plot(cumulative_heads_ratio)
plt.plot([0, 10000], [0.51, 0.51], "k--", linewidth=2, label="51%")
plt.plot([0, 10000], [0.5, 0.5], "k-", label="50%")
plt.xlabel("Number of coin tosses")
plt.ylabel("Heads ratio")
plt.legend(loc="lower right")
plt.axis([0, 10000, 0.42, 0.58])
plt.show()

## Voto por Mayoria

El algoritmo que vamos a implementar en esta sección nos permitirá combinar diferentes algoritmos de clasificación asociados con pesos individuales por confianza. Nuestro objetivo es construir un metaclasificador más fuerte que equilibre las debilidades de los clasificadores individuales en un conjunto de datos en particular. En términos matemáticos más precisos, podemos escribir el voto de la mayoría ponderada de la siguiente manera:

$$ \hat{y}=arg max \sum^{m}_{i=1}w_{j}X_{A}(C_{j}(x)=i)$$

Donde $w_{j}$ es el peso del clasificador $C_{j}$. $\hat{y}$ es la predicción del ensamble; A es el conjunto de etiquetas de clase únicas;$X_{A}$ es la función característica o función indicadora, que devuelve 1 si la clase predicha del clasificador j-ésimo coincide con $i (C_j(x) = i)$. 


In [ ]:
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
import numpy as np
import operator

En la siguiente función se genera el voto de los modelos que utilizarán de acuerdo a la predicción.

In [ ]:
class MajorityVoteClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, classifiers, vote='classlabel', weights=None):
        self.classifiers = classifiers
        self.named_classifiers = {
            key: value for key,
            value in _name_estimators(classifiers)
        }
        self.vote = vote
        self.weights = weights
    def fit(self, X, y):
        if self.vote not in ('probability', 'classlabel'):
            raise ValueError(f"vote must be 'probability'"f"or 'classlabel'"f"; got (vote={self.vote})")
        if self.weights and len(self.weights) != len(self.classifiers):
            raise ValueError(f'Number of classifiers and'f' weights must be equal'f'; got {len(self.weights)} weights,'f' {len(self.classifiers)} classifiers') # Use LabelEncoder to ensure class labels start
# with 0, which is important for np.argmax # call in self.predict
        self.lablenc_ = LabelEncoder() 
        self.lablenc_.fit(y)
        self.classes_ = self.lablenc_.classes_
        self.classifiers_ = []
        for clf in self.classifiers:
            fitted_clf = clone(clf).fit(X,
                           self.lablenc_.transform(y))
            self.classifiers_.append(fitted_clf)
        return self
    def predict(self, X):
        if self.vote == 'probability':
            maj_vote = np.argmax(self.predict_proba(X), axis=1) 
        else: # 'classlabel' vote
            predictions = np.asarray([
            clf.predict(X) for clf in self.classifiers_
            ]).T
            maj_vote = np.apply_along_axis(
                lambda x: np.argmax(
                    np.bincount(x, weights=self.weights)
                ),
                axis=1, arr=predictions
            )
        maj_vote = self.lablenc_.inverse_transform(maj_vote)
        return maj_vote
    def predict_proba(self, X):
        probas = np.asarray([clf.predict_proba(X)
                         for clf in self.classifiers_])
        avg_proba = np.average(probas, axis=0,
                           weights=self.weights)
        return avg_proba
    def get_params(self, deep=True):
        if not deep:
            return super().get_params(deep=False)
        else:
            out = self.named_classifiers.copy()
            for name, step in self.named_classifiers.items():
                for key, value in step.get_params(
                    deep=True).items():
                    out[f'{name}__{key}'] = value
            return out
    
    

Con esta función se genera la predicción del método de ensamble combinando los diferentes clasificadores.

Dado que ya estamos familiarizados con las técnicas para cargar conjuntos de datos desde archivos CSV, tomaremos un atajo y cargaremos el conjunto de datos de Iris desde el módulo de conjuntos de datos de scikit-learn. Además, solo seleccionaremos dos características, el ancho del sépalo y la longitud del pétalo, para que la tarea de clasificación sea más desafiante con fines ilustrativos. Aunque nuestro MajorityVoteClassifier generaliza a problemas multiclase, solo clasificaremos ejemplos de flores de las clases Iris-versicolor e Iris-virginica, con las que calcularemos el ROC AUC más adelante. El código es el siguiente:

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import LabelEncoder
iris = datasets.load_iris()
X, y = iris.data[50:, [1, 2]], iris.target[50:]
le = LabelEncoder()
y = le.fit_transform(y)
X_train, X_test, y_train, y_test =train_test_split(X, y,test_size=0.5,random_state=1,stratify=y)

Para este modelo usaremos la regresión logística, un árbol de desición y KNN

In [ ]:
from sklearn.model_selection import cross_val_score 
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline
import numpy as np
clf1 = LogisticRegression(penalty='l2',C=0.001,solver='lbfgs',random_state=1) 
clf2 = DecisionTreeClassifier(max_depth=1,criterion='entropy',random_state=0) 
clf3 = KNeighborsClassifier(n_neighbors=1,p=2, metric='minkowski') 
pipe1 = Pipeline([['sc', StandardScaler()],['clf', clf1]])
pipe3 = Pipeline([['sc', StandardScaler()],['clf', clf3]])
clf_labels = ['Logistic regression', 'Decision tree', 'KNN'] 
print('10-fold cross validation:\n')
for clf, label in zip([pipe1, clf2, pipe3], clf_labels):
    scores = cross_val_score(estimator=clf,X=X_train,y=y_train,cv=10, scoring='roc_auc')
    print(f'ROC AUC: {scores.mean():.2f} 'f'(+/- {scores.std():.2f}) [{label}]')

Quizás se pregunte por qué entrenamos la regresión logística y el clasificador de vecinos más cercanos como parte de una canalización. La razón detrás de esto es que, tanto la regresión logística como los algoritmos de k vecinos más cercanos (usando la métrica de distancia euclidiana) no son invariantes en escala, en contraste con los árboles de decisión. Aunque las características de Iris se miden todas en la misma escala (cm), es un buen hábito trabajar con características estandarizadas.

In [ ]:
mv_clf = MajorityVoteClassifier(classifiers=[pipe1, clf2, pipe3])
clf_labels += ['Majority voting']
all_clf = [pipe1, clf2, pipe3, mv_clf]
for clf, label in zip(all_clf, clf_labels):
    scores = cross_val_score(estimator=clf,X=X_train,y=y_train,cv=10,scoring="roc_auc")
    print(f'ROC AUC: {scores.mean():.2f} 'f'(+/- {scores.std():.2f}) [{label}]')

### Eavluación del modelo

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
colors = ['black', 'orange', 'blue', 'green']
linestyles = [':', '--', '-.', '-']
for clf, label, clr, ls in zip(all_clf, clf_labels, colors, linestyles): # assuming the label of the positive class is 1 y_pred = clf.fit(X_train,
    y_pred =clf.fit(X_train, y_train).predict_proba(X_test)[:, 1] 
    fpr, tpr, thresholds = roc_curve(y_true=y_test,y_score=y_pred)
    roc_auc = auc(x=fpr, y=tpr)
    plt.plot(fpr,tpr,color=clr,linestyle=ls,label =f'{label} (auc = {roc_auc:.2f})')
    
plt.legend(loc='lower right')  
plt.plot([0, 1], [0, 1], linestyle='--',color='gray',linewidth=2)
plt.xlim([-0.1, 1.1])
plt.ylim([-0.1, 1.1])
plt.grid(alpha=0.5)
plt.xlabel('False positive rate (FPR)') 
plt.ylabel('True positive rate (TPR)') 
plt.show()

In [ ]:
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train) 
from itertools import product
x_min = X_train_std[:, 0].min() - 1
x_max = X_train_std[:, 0].max() + 1
y_min = X_train_std[:, 1].min() - 1
y_max = X_train_std[:, 1].max() + 1

In [ ]:
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),np.arange(y_min, y_max, 0.1)) 
f, axarr = plt.subplots(nrows=2, ncols=2,sharex='col',sharey='row',figsize=(7, 5))
for idx, clf, tt in zip(product([0, 1], [0, 1]), all_clf, clf_labels):
    clf.fit(X_train_std, y_train)
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    axarr[idx[0], idx[1]].contourf(xx, yy, Z, alpha=0.3)
    axarr[idx[0], idx[1]].scatter(X_train_std[y_train==0, 0],X_train_std[y_train==0, 1],c='blue',marker='^',s=50)
    axarr[idx[0], idx[1]].scatter(X_train_std[y_train==1, 0],X_train_std[y_train==1, 1],c='green',marker='o',s=50)
    axarr[idx[0], idx[1]].set_title(tt)
plt.text(-3.5, -5.,s='Sepal width [standardized]',ha='center', va='center', fontsize=12) 
plt.text(-12.5, 4.5,s='Petal length [standardized]',ha='center', va='center',fontsize=12, rotation=90) 
plt.show()

## Random Forest

En esta sección, vamos a analizar algunos de estos algoritmos, en especial, los de taxonomía lineal y no lineal. En cuanto a la taxonomía de conjunto o ensamblados, como los tipo boosting y bagging, los veremos posteriormente cuando ya tengamos una base sólida de estos primeros. Cada algoritmo será presentado desde dos perspectivas:
* El paquete y la función utilizados para entrenar y hacer predicciones.
* Las configuraciones en el paquete scikit-learn para cada algoritmo, estos es, la configuración de sus hiperparámetros.

Si todos los clasificadores son capaces de estimar las probabilidades de clase (es decir, todos tienen un método predict_proba()), entonces puede decirle a Scikit-Learn que prediga la clase con la mayor probabilidad de clase, promediada entre todos los clasificadores individuales. Esto se llama votación suave. A menudo logra un mayor rendimiento que la votación dura porque da más peso a los votos de alta confianza. Lo único que hay que hacer es sustituir voting="hard" por voting="soft" y asegurarse de que todos los clasificadores pueden estimar las probabilidades de clase. Este no es el caso de la clase SVC por defecto, por lo que debe establecer su hiperparámetro de probabilidad en True (esto hará que la clase SVC utilice la validación cruzada para estimar las probabilidades de clase, ralentizando el entrenamiento, y añadirá un método predict_proba()). Si modifica el código anterior para utilizar la votación suave, verá que el clasificador de votación alcanza una precisión superior al 91,2%.

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import plot_tree 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import plot_confusion_matrix 
from sklearn.metrics import plot_confusion_matrix 
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### Bagging y Passing

Una forma de obtener un conjunto diverso de clasificadores es utilizar algoritmos de entrenamiento muy diferentes, como acabamos de comentar. Otro enfoque es utilizar el mismo algoritmo de entrenamiento para cada predictor y entrenarlos en diferentes subconjuntos aleatorios del conjunto de entrenamiento. Cuando el muestreo se realiza con reemplazo, este método se denomina bagging (abreviatura de bootstrap aggregating ). Cuando el muestreo se realiza sin reemplazo, se denomina 'pasting'. En otras palabras, tanto el bagging como el pasting permiten muestrear las instancias de entrenamiento varias veces entre múltiples predictores, pero sólo el bagging permite muestrear las instancias de entrenamiento varias veces para el mismo predictor. 

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, random_state=42)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

Una vez entrenados todos los predictores, el conjunto puede hacer una predicción para una nueva instancia simplemente agregando las predicciones de todos los predictores. La función de agregación suele ser el modo estadístico (es decir, la predicción más frecuente, al igual que un clasificador de voto duro) para la clasificación, o la media para la regresión. Cada predictor individual tiene un sesgo mayor que si se entrenara con el conjunto de entrenamiento original, pero la agregación reduce tanto el sesgo como la varianza. En general, el resultado neto es que el conjunto tiene un sesgo similar pero una varianza menor que un único predictor entrenado en el conjunto de entrenamiento original.

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

In [ ]:
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_tree))

Los predictores pueden ser entrenados en paralelo, a través de diferentes núcleos de CPU o incluso diferentes servidores. Del mismo modo, las predicciones pueden realizarse en paralelo. Esta es una de las razones por las que el bagging y el pasting son métodos tan populares.

El parámetro n_jobs indica a Scikit-Learn el número de núcleos de CPU que debe utilizar para el entrenamiento y las predicciones (-1 indica a Scikit-Learn que debe utilizar todos los núcleos disponibles)

In [ ]:
from matplotlib.colors import ListedColormap

def plot_decision_boundary(clf, X, y, axes=[-1.5, 2.45, -1, 1.5], alpha=0.5, contour=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if contour:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", alpha=alpha)
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", alpha=alpha)
    plt.axis(axes)
    plt.xlabel(r"$x_1$", fontsize=18)
    plt.ylabel(r"$x_2$", fontsize=18, rotation=0)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(10,4), sharey=True)
plt.sca(axes[0])
plot_decision_boundary(tree_clf, X, y)
plt.title("Decision Tree", fontsize=14)
plt.sca(axes[1])
plot_decision_boundary(bag_clf, X, y)
plt.title("Decision Trees with Bagging", fontsize=14)
plt.ylabel("")

plt.show()

### Random Forest 

El algoritmo Random Forest introduce una aleatoriedad adicional en el crecimiento de los árboles; en lugar de buscar la mejor característica al dividir un nodo, busca la mejor característica entre un subconjunto aleatorio de características. El algoritmo da lugar a una mayor diversidad de árboles, lo que (de nuevo) supone un mayor sesgo a cambio de una menor varianza, lo que generalmente da lugar a un modelo globalmente mejor.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

Random Forest es un conjunto de Árboles de Decisión, generalmente entrenado a través del método de bagging (o a veces de Pasting), normalmente con max_samples establecido al tamaño del conjunto de entrenamiento. En lugar de construir un BaggingClassifier y pasarle un DecisionTreeClassifier, se puede utilizar la clase RandomForestClassifier, que es más conveniente y está optimizada para los Árboles de Decisión (de forma similar, hay una clase RandomForestRegressor para tareas de regresión). 

El siguiente BaggingClassifier es aproximadamente equivalente al anterior RandomForestClassifier:

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="sqrt", max_leaf_nodes=16),
    n_estimators=500, random_state=42)

In [ ]:
np.sum(y_pred == y_pred_rf) / len(y_pred)  # predicciones muy similares

# Caso Clasificación. Credit Risk

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df_credit = pd.read_csv('data_credit')
df_credit=df_credit[['grade','emp_length_n', 'dti_n', 'revenue', 'loan_amnt', 'fico_n',
       'purpose_car', 'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_educational', 'purpose_home_improvement', 'purpose_house',
       'purpose_major_purchase', 'purpose_medical', 'purpose_moving',
       'purpose_other', 'purpose_renewable_energy',
       'purpose_small_business', 'purpose_vacation', 'purpose_wedding',
       'home_ownership_n_MORTGAGE', 'home_ownership_n_OTHER',
       'home_ownership_n_OWN', 'home_ownership_n_RENT']]
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
le = LabelEncoder()
df_credit['grade'] = le.fit_transform(df_credit['grade'])
df_credit['grade'].value_counts(normalize=True, sort=True, ascending=True)
df_credit=df_credit.groupby('grade', group_keys=False).apply(lambda x: x.sample(frac=0.05, random_state=123))
df_credit['grade'].value_counts(normalize=True, sort=True, ascending=True)
df_credit['grade'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder


X_train, X_test, y_train, y_test = train_test_split(df_credit.drop(labels=['grade', ], axis=1),df_credit['grade'], test_size=0.3, random_state=123, stratify=df_credit['grade'])

#X_train_s = MinMaxScaler(X_train)
scaler = MinMaxScaler()
X_train_s=scaler.fit_transform(X_train)
X_test_s=scaler.fit_transform(X_test)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import pandas as pd
from sklearn.compose import ColumnTransformer
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
tf.random.set_seed(123)

neurons=5
optimizer='adam'

def model_ann(neurons=neurons, optimizer=optimizer):
  # creating the layers of the ANN
  ann = tf.keras.models.Sequential()
  ann.add(tf.keras.layers.Dense(units=neurons, input_dim=len(X_train.keys()), activation='relu'))
  ann.add(tf.keras.layers.Dense(units=neurons, activation='relu'))
  ann.add(tf.keras.layers.Dense(units=7, activation='softmax'))
  # Compile model
  ann.compile(optimizer=optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  return ann

model=KerasClassifier(build_fn=model_ann, verbose=0)

params={'batch_size':[50, 32], 
        'nb_epoch':[20, 50],
        'neurons':[15, 30],
       'optimizer':['rmsprop', 'adam'],}

'''activations = ['tanh','relu','sigmoid']
initializers = ['glorot_uniform', 'normal', 'uniform']'''

grid = GridSearchCV(estimator=model, param_grid=params, cv=3)
grid_result = grid.fit(X_train_s, y_train, verbose=0)

### Optimización de hiperparámetros. Estadísticas. Identificación del mejor conjunto de hiperparámetros

In [ ]:
print("El mejor accuracy: {}\nY la mejor combinación de hiperparámetros: {}".format(grid_result.best_score_, 
                             grid_result.best_params_))

In [ ]:
import pandas as pd
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
d_hhiper=pd.DataFrame(params)
d_hhiper['Mean']=means
d_hhiper['Std. Dev']=stds
d_hhiper.sort_values(by='Mean', ascending=False).head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
param_ = ['batch_size','nb_epoch','neurons','optimizer']

fig, ax = plt.subplots(2,2,figsize=(10,6), squeeze=False)
ax = ax.ravel()
for i in range(4):
    ax[i].set_title('Distribution of mean accuracy with {}'.format(param_[i]))
    sns.boxplot(x=param_[i],y='Mean',data=d_hhiper,ax=ax[i])
fig.tight_layout(pad=1.5)

### Diferentes algoritmos de modelación vs la "mejor" red. Desempeño - Stratified CV

In [ ]:
#Comparar modelos
import matplotlib.pyplot as plt
from sklearn import model_selection
from keras.wrappers.scikit_learn import KerasClassifier
import pandas
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
tf.random.set_seed(123)

neurons=grid_result.best_params_['neurons']
optimizer=grid_result.best_params_['optimizer']

epochs=grid_result.best_params_['nb_epoch']
batch_size=grid_result.best_params_['batch_size']

model_ann_f = KerasClassifier(build_fn=model_ann, epochs=epochs, batch_size=batch_size, verbose=0)

seed = 221
models = []
models.append(('MLR', LogisticRegression(random_state = 123)))
models.append(('CART', DecisionTreeClassifier(random_state = 123)))
models.append(('RF', RandomForestClassifier(random_state = 123)))
models.append(('ANN', model_ann_f))
# evaluate each model in turn
results_c = []
names_c = []
scoring = 'accuracy'
for name, model in models:
    kfold = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train_s, y_train, cv=kfold, scoring=scoring)
    results_c.append(cv_results)
    names_c.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('CV-ACC Comparación de algoritmos')
ax = fig.add_subplot(111)
plt.boxplot(results_c)
ax.set_xticklabels(names_c)
plt.show()

In [ ]:
#Comparar modelos. Entrenamiento con DataTrain completo y evaluación con DataTest.
import matplotlib.pyplot as plt
from sklearn import model_selection
from keras.wrappers.scikit_learn import KerasClassifier
import pandas
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
tf.random.set_seed(123)

neurons=grid_result.best_params_['neurons']
optimizer=grid_result.best_params_['optimizer']
epochs=grid_result.best_params_['nb_epoch']
batch_size=grid_result.best_params_['batch_size']

model_ann_f = model_ann(neurons, optimizer)
model_ann_f.fit(X_train_s, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
test_loss, test_acc = model_ann_f.evaluate(X_test_s, y_test, verbose=0)

seed = 221
models = []
models.append(('MLR', LogisticRegression(random_state = 123)))
models.append(('CART', DecisionTreeClassifier(random_state = 123)))
models.append(('RF', RandomForestClassifier(random_state = 123)))
# evaluate each model in turn
results_t = []
names_t = []
scoring = 'accuracy'
for name, model in models:
    model.fit(X_train_s, y_train)
    results_m=accuracy_score(y_test, model.predict(X_test_s))
    results_t.append(results_m)
    names_t.append(name)
    msg = "%s: %f" % (name, results_m)
    print(msg)
print("%s: %f" % ('ANN', test_acc))

results_t.append(test_acc)
names_t.append('ANN')


# boxplot algorithm comparison
fig = plt.figure()
sns.set_style('darkgrid')
fig.suptitle('Comparación de algoritmos en DataTest con ACC')
ax = fig.add_subplot(111)
sns.barplot(names_t, results_t)
ax.bar_label(ax.containers[0], label_type='edge')
plt.show()

In [ ]:
# Predicción individual 
import numpy as np
new_register = np.expand_dims(X_test_s[4,:],0)

predictions_single = model_ann_f.predict(new_register)
print(predictions_single)

In [ ]:
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

print('Predicción de Grade:', np.argmax(predictions_single[0]),";", 'Real Grade:', y_test.iloc[4])

def plot_value_array(i, predictions, true_y):
  predictions, true_y = predictions, true_y.iloc[i]
  plt.grid(False)
  plt.xticks(range(7))
  plt.yticks([])
  thisplot = plt.bar(range(7), predictions, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions)
  thisplot[predicted_label].set_color('red')
  thisplot[true_y].set_color('blue')

plot_value_array(4, predictions_single[0], y_test)
_ = plt.xticks(range(7), class_names, rotation=45)

## Libreria Sklearn

En esta sección, vamos a analizar algunos de estos algoritmos, en especial, los de taxonomía lineal y no lineal. En cuanto a la taxonomía de conjunto o ensamblados, como los tipo boosting y bagging, los veremos posteriormente cuando ya tengamos una base sólida de estos primeros. Cada algoritmo será presentado desde dos perspectivas:

* El paquete y la función utilizados para entrenar y hacer predicciones.
* Las configuraciones en el paquete scikit-learn para cada algoritmo, estos es, la configuración de sus hiperparámetros.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=10, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')

In [ ]:
voting_clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred =clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

Si todos los clasificadores son capaces de estimar las probabilidades de clase (es decir, todos tienen un método predict_proba()), entonces puede decirle a Scikit-Learn que prediga la clase con la mayor probabilidad de clase, promediada entre todos los clasificadores individuales. Esto se llama votación suave. A menudo logra un mayor rendimiento que la votación dura porque da más peso a los votos de alta confianza. Lo único que hay que hacer es sustituir voting="hard" por voting="soft" y asegurarse de que todos los clasificadores pueden estimar las probabilidades de clase. Este no es el caso de la clase SVC por defecto, por lo que debe establecer su hiperparámetro de probabilidad en True (esto hará que la clase SVC utilice la validación cruzada para estimar las probabilidades de clase, ralentizando el entrenamiento, y añadirá un método predict_proba()). Si modifica el código anterior para utilizar la votación suave, verá que el clasificador de votación alcanza una precisión superior al 91,2%.

In [ ]:
log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", probability=True, random_state=42) # observe la modificación si necesitamos calcular las probabilidades asociadas

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft')
voting_clf.fit(X_train, y_train)

In [ ]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred =clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))